In [12]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/')

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

import os
from pipeline_config import *


In [14]:
#Seeds for reproducability:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [15]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/test_predictions.pickle', 'rb') as f:
    prediction_dic = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/test_predictions_1.pickle', 'rb') as f:
    prediction_dic_no_1 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/test_predictions_2.pickle', 'rb') as f:
    prediction_dic_no_2 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/test_predictions_3.pickle', 'rb') as f:
    prediction_dic_no_3 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/test_predictions_1_2.pickle', 'rb') as f:
    prediction_dic_no_1_2 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/pickle_files/bi_directional_norway_dataset_dict.pickle', 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [16]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
    #for key in ['All items']:
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_70654/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_70654/1267119108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_70654/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [17]:
def get_df_with_predictions(prediction_dic, prediction_dic_1: dict, prediction_dic_2: dict, prediction_dic_3: dict, prediction_dic_1_2: dict,  dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        predictions_1 = prediction_dic_1[key]
        predictions_2 = prediction_dic_2[key]
        predictions_3 = prediction_dic_3[key]
        predictions_1_2 = prediction_dic_1_2[key]

        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        prediction_1_df =  pd.DataFrame(predictions_1.flatten().detach().numpy())
        prediction_1_df.rename(columns = {0: 'Prediction_No_1'}, inplace=True)
        prediction_2_df =  pd.DataFrame(predictions_2.flatten().detach().numpy())
        prediction_2_df.rename(columns = {0: 'Prediction_No_2'}, inplace=True)
        prediction_3_df =  pd.DataFrame(predictions_3.flatten().detach().numpy())
        prediction_3_df.rename(columns = {0: 'Prediction_No_3'}, inplace=True)
        prediction_1_2_df =  pd.DataFrame(predictions_1_2.flatten().detach().numpy())
        prediction_1_2_df.rename(columns = {0: 'Prediction_No_1_2'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        

        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df, prediction_1_df, prediction_2_df, prediction_3_df, prediction_1_2_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, prediction_dic_no_1, prediction_dic_no_2, prediction_dic_no_3, prediction_dic_no_1_2, test_dict)

In [18]:
# sanity check
all_categories = list(all_data_test_dict.keys())
len(all_categories)

52

In [19]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    mse_lst_1 = []
    mse_lst_2 = []
    mse_lst_3 = []
    mse_lst_1_2 = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_actual = df_predictions['Actual'].values

        y_pred = df_predictions['Prediction'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)

        y_pred_1 = df_predictions['Prediction_No_1'].values
        mse_1 = mean_squared_error(y_pred_1, y_actual)
        mse_lst_1.append(mse_1)

        y_pred_2 = df_predictions['Prediction_No_2'].values
        mse_2 = mean_squared_error(y_pred_2, y_actual)
        mse_lst_2.append(mse_2)

        y_pred_3 = df_predictions['Prediction_No_3'].values
        mse_3 = mean_squared_error(y_pred_3, y_actual)
        mse_lst_3.append(mse_3)

        y_pred_1_2 = df_predictions['Prediction_No_1_2'].values
        mse_1_2 = mean_squared_error(y_pred_1_2, y_actual)
        mse_lst_1_2.append(mse_1_2)

    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    rmse_list_1 = list(map(np.sqrt,mse_lst_1))
    avg_rmse_1 = np.mean(rmse_list_1)
    rmse_std_1 = np.std(rmse_list_1)

    rmse_list_2 = list(map(np.sqrt,mse_lst_2))
    avg_rmse_2 = np.mean(rmse_list_2)
    rmse_std_2 = np.std(rmse_list_2)

    rmse_list_3 = list(map(np.sqrt,mse_lst_3))
    avg_rmse_3 = np.mean(rmse_list_3)
    rmse_std_3 = np.std(rmse_list_3)
    
    rmse_list_1_2 = list(map(np.sqrt,mse_lst_1_2))
    avg_rmse_1_2 = np.mean(rmse_list_1_2)
    rmse_std_1_2 = np.std(rmse_list_1_2)

    print(f'RMSE:  {avg_rmse}')
    print(f'RMSE - No Loss Term 1:  {avg_rmse_1}, difference = {avg_rmse_1-avg_rmse}')
    print(f'RMSE - No Loss Term 2:  {avg_rmse_2}, difference = {avg_rmse_2-avg_rmse}')
    print(f'RMSE - No Loss Term 3:  {avg_rmse_3}, difference = {avg_rmse_3-avg_rmse}')
    print(f'RMSE - No Loss Term 1+2:  {avg_rmse_1_2}, difference = {avg_rmse_1_2-avg_rmse}')
    print('-----------------------------------------------------------------------------')
    print(f'MSE std:  {rmse_std}')
    print(f'MSE std - No Loss Term 1:  {rmse_std_1}')
    print(f'MSE std - No Loss Term 2:  {rmse_std_2}')
    print(f'MSE std - No Loss Term 3:  {rmse_std_3}')
    print(f'MSE std - No Loss Term 1+2:  {rmse_std_1_2}')

    return avg_rmse, rmse_std, avg_rmse_1,rmse_std_1, avg_rmse_2,rmse_std_2, avg_rmse_3,rmse_std_3, avg_rmse_1_2,rmse_std_1_2

avg_rmse(all_data_test_dict)

RMSE:  1.3860573530789315
RMSE - No Loss Term 1:  1.358135656452388, difference = -0.027921696626543424
RMSE - No Loss Term 2:  1.3860573530789315, difference = 0.0
RMSE - No Loss Term 3:  1.3602008216558805, difference = -0.025856531423051
RMSE - No Loss Term 1+2:  1.358135656452388, difference = -0.027921696626543424
-----------------------------------------------------------------------------
MSE std:  1.293176771848149
MSE std - No Loss Term 1:  1.3086563940542923
MSE std - No Loss Term 2:  1.293176771848149
MSE std - No Loss Term 3:  1.2629430392029632
MSE std - No Loss Term 1+2:  1.3086563940542923


(1.3860573530789315,
 1.293176771848149,
 1.358135656452388,
 1.3086563940542923,
 1.3860573530789315,
 1.293176771848149,
 1.3602008216558805,
 1.2629430392029632,
 1.358135656452388,
 1.3086563940542923)

In [20]:
df_predictions = all_data_test_dict['All-items']
y_actual = df_predictions['Actual'].values

y_pred = df_predictions['Prediction'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)
print(f'Headline RMSE: {rmse}')

y_pred_1 = df_predictions['Prediction_No_1'].values
mse_1 = mean_squared_error(y_pred_1, y_actual)
rmse_1 = np.sqrt(mse_1)
print(f'Headline RMSE - No Loss Term 1: {rmse_1}, difference = {rmse_1-rmse}')

y_pred_2 = df_predictions['Prediction_No_2'].values
mse_2 = mean_squared_error(y_pred_2, y_actual)
rmse_2 = np.sqrt(mse_2)
print(f'Headline RMSE - No Loss Term 2: {rmse_2}, difference = {rmse_2-rmse}')

y_pred_3 = df_predictions['Prediction_No_3'].values
mse_3 = mean_squared_error(y_pred_3, y_actual)
rmse_3 = np.sqrt(mse_3)
print(f'Headline RMSE - No Loss Term 3: {rmse_3}, difference = {rmse_3-rmse}')

y_pred_1_2 = df_predictions['Prediction_No_1_2'].values
mse_1_2 = mean_squared_error(y_pred_1_2, y_actual)
rmse_1_2 = np.sqrt(mse_1_2)
print(f'Headline RMSE - No Loss Term 1+2: {rmse_1_2}, difference = {rmse_1_2-rmse}')

Headline RMSE: 0.5165681676632432
Headline RMSE - No Loss Term 1: 0.5165681676632432, difference = 0.0
Headline RMSE - No Loss Term 2: 0.5165681676632432, difference = 0.0
Headline RMSE - No Loss Term 3: 0.5057512603115635, difference = -0.010816907351679728
Headline RMSE - No Loss Term 1+2: 0.5165681676632432, difference = 0.0


In [21]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    corr_dict_1 = {}
    corr_dict_2 = {}
    corr_dict_3 = {}
    corr_dict_1_2 = {}

    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_actual = df_predictions['Actual'].values

        y_pred = df_predictions['Prediction'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr

        y_pred_1 = df_predictions['Prediction_No_1'].values
        corr_1 = stats.pearsonr(y_pred_1,y_actual)[0]
        corr_dict_1[key] =  corr_1

        y_pred_2 = df_predictions['Prediction_No_2'].values
        corr_2 = stats.pearsonr(y_pred_2,y_actual)[0]
        corr_dict_2[key] =  corr_2

        y_pred_3 = df_predictions['Prediction_No_3'].values
        corr_3 = stats.pearsonr(y_pred_3,y_actual)[0]
        corr_dict_3[key] =  corr_3

        y_pred_1_2 = df_predictions['Prediction_No_1_2'].values
        corr_1_2 = stats.pearsonr(y_pred_1_2,y_actual)[0]
        corr_dict_1_2[key] =  corr_1_2

    total_corr = sum(corr_dict.values())
    total_corr_1 = sum(corr_dict_1.values())
    total_corr_2 = sum(corr_dict_2.values())
    total_corr_3 = sum(corr_dict_3.values())
    total_corr_1_2 = sum(corr_dict_1_2.values())
    
    print(f'total_corr = {total_corr}')
    print(f'total corr - No Loss Term 1: {total_corr_1}, difference = {total_corr-total_corr_1}')
    print(f'total corr - No Loss Term 2: {total_corr_2}, difference = {total_corr-total_corr_2}')
    print(f'total corr - No Loss Term 3: {total_corr_3}, difference = {total_corr-total_corr_3}')
    print(f'total corr - No Loss Term 1+2: {total_corr_1_2}, difference = {total_corr-total_corr_1_2}')
    return total_corr, total_corr_1, total_corr_2, total_corr_3, total_corr_1_2

total_corr(all_data_test_dict)

total_corr = 23.041972415761112
total corr - No Loss Term 1: 24.89460064996252, difference = -1.8526282342014078
total corr - No Loss Term 2: 23.041972415761112, difference = 0.0
total corr - No Loss Term 3: 23.184219559786648, difference = -0.14224714402553573
total corr - No Loss Term 1+2: 24.89460064996252, difference = -1.8526282342014078


(23.041972415761112,
 24.89460064996252,
 23.041972415761112,
 23.184219559786648,
 24.89460064996252)

In [22]:
def total_corr_all_items(all_data_test_dict):
    corr_dict = {}
    corr_dict_1 = {}
    corr_dict_2 = {}
    corr_dict_3 = {}
    corr_dict_1_2 = {}

    for key in ['All-items']:
        df_predictions = all_data_test_dict[key]
        y_actual = df_predictions['Actual'].values

        y_pred = df_predictions['Prediction'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr

        y_pred_1 = df_predictions['Prediction_No_1'].values
        corr_1 = stats.pearsonr(y_pred_1,y_actual)[0]
        corr_dict_1[key] =  corr_1

        y_pred_2 = df_predictions['Prediction_No_2'].values
        corr_2 = stats.pearsonr(y_pred_2,y_actual)[0]
        corr_dict_2[key] =  corr_2

        y_pred_3 = df_predictions['Prediction_No_3'].values
        corr_3 = stats.pearsonr(y_pred_3,y_actual)[0]
        corr_dict_3[key] =  corr_3

        y_pred_1_2 = df_predictions['Prediction_No_1_2'].values
        corr_1_2 = stats.pearsonr(y_pred_1_2,y_actual)[0]
        corr_dict_1_2[key] =  corr_1_2

    total_corr = sum(corr_dict.values())
    total_corr_1 = sum(corr_dict_1.values())
    total_corr_2 = sum(corr_dict_2.values())
    total_corr_3 = sum(corr_dict_3.values())
    total_corr_1_2 = sum(corr_dict_1_2.values())
    
    print(f'Headline Corr = {total_corr}')
    print(f'Headline corr - No Loss Term 1: {total_corr_1}, difference = {total_corr-total_corr_1}')
    print(f'Headline corr - No Loss Term 2: {total_corr_2}, difference = {total_corr-total_corr_2}')
    print(f'Headline corr - No Loss Term 3: {total_corr_3}, difference = {total_corr-total_corr_3}')
    print(f'Headline corr - No Loss Term 1+2: {total_corr_1_2}, difference = {total_corr-total_corr_1_2}')
    return total_corr, total_corr_1, total_corr_2, total_corr_3,total_corr_1_2

total_corr_all_items(all_data_test_dict)

Headline Corr = 0.44916427360074396
Headline corr - No Loss Term 1: 0.44916427360074396, difference = 0.0
Headline corr - No Loss Term 2: 0.44916427360074396, difference = 0.0
Headline corr - No Loss Term 3: 0.40296791849670166, difference = 0.0461963551040423
Headline corr - No Loss Term 1+2: 0.44916427360074396, difference = 0.0


(0.44916427360074396,
 0.44916427360074396,
 0.44916427360074396,
 0.40296791849670166,
 0.44916427360074396)